In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.

In [3]:
import machine

a34 = machine.ADC(machine.Pin(34))
a34.atten(machine.ADC.ATTN_11DB)
a35 = machine.ADC(machine.Pin(35))
a35.atten(machine.ADC.ATTN_11DB)


In [12]:
import time

ads = [a34,a35]
def measure34sub(numberofseconds, samplespersecond, subsamplestoaverage, compensationratio):
    ticks0 = time.ticks_ms()
    
    # compensationratio is used to shorten the sleep cycle so that we hit the right time durations
    # (by sleeping slightly less than we should because the rest of the code is slow)
    microsecondstep = int(1/samplespersecond/subsamplestoaverage*1000000*compensationratio)
    
    # one second per line
    for i in range(numberofseconds):
        
        # readings per second
        for j in range(samplespersecond):
            
            # number of samples averaged per (1/samplespersecond)^th of a second for a reading
            samplesums = [0]*len(ads)
            for k in range(subsamplestoaverage):
                
                # make a reading from each of the ADC
                for i, a in enumerate(ads):
                    samplesums[i] += a.read()
                time.sleep_us(microsecondstep)
            print(",".join(str(int(samplesum/subsamplestoaverage+0.5))  for samplesum in samplesums), end=" ")
        print()
    return (time.ticks_ms() - ticks0)/1000
    

In [22]:
%capture --quiet testpulsemulti50avg.txt
elapsedtime = measure34sub(10, 50, 10, 0.7)


10 lines captured

In [21]:
print(elapsedtime)

10.348
